In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
from google.colab import files

In [35]:
pd.options.display.float_format = "{:.6g}".format

In [70]:
uploaded = files.upload()
uploaded_paths = list(uploaded.keys())
print("Uploaded files:", uploaded_paths)

Saving 202501_기준금속.xlsx to 202501_기준금속 (1).xlsx
Uploaded files: ['202501_기준금속 (1).xlsx']


In [82]:
# 자동 검출된 컬럼명 확인 후 True 반영
CONFIRM_DETECTED_COLUMNS = True

# 전처리 옵션
NEGATIVE_TO_NAN = True   # 음수값을 NaN으로 처리

# 시각화 및 리포트 파일명
RAW_REPORT_PATH   = "Step1_QA_Report_raw.xlsx"    # 전처리 전 QA 리포트
CLEAN_REPORT_PATH = "Step1_QA_Report_clean.xlsx"  # 전처리 후 QA 리포트
CLEAN_DATA_PATH   = "Cleaned_dataset.xlsx"        # 전처리 후 데이터(엑셀)
CLEAN_DATA_CSV    = "Cleaned_dataset.csv"         # 전처리 후 데이터(CSV)

In [83]:
# 컬럼 자동검출 함수
def detect_columns(df: pd.DataFrame):
    # 공백 제거
    df = df.copy()
    df.columns = [str(c).strip() for c in df.columns]

    # 시간 컬럼
    time_candidates = [c for c in df.columns
                       if any(k in c.lower() for k in ["pump-begin", "date", "datetime", "time", "측정일시"])]
    time_col = time_candidates[0] if len(time_candidates) > 0 else None

    # PM2.5 컬럼
    pm25_candidates = [c for c in df.columns
                       if ("pm2.5" in c.lower())
                       or ("con(ug/m3)" in c.lower())
                       or ("ug/m3" in c.lower() and "con" in c.lower())]
    pm25_col = pm25_candidates[0] if len(pm25_candidates) > 0 else None

    # 금속 단위 패턴
    metal_markers = ["(ng/m3)", "(ng/㎥)", "ng/m3", "ng/㎥"]
    metal_cols = [c for c in df.columns if any(m.lower() in c.lower() for m in metal_markers)]

    # 숫자형 후보
    numeric_cols = []
    for c in df.columns:
        ser = pd.to_numeric(df[c], errors="coerce")
        if ser.notna().mean() >= 0.7:
            numeric_cols.append(c)

    return {
        "time_col": time_col,
        "pm25_col": pm25_col,
        "metal_cols": metal_cols,
        "numeric_cols": numeric_cols}

In [84]:
# 컬럼 자동 검출 및 확인게이트
path = uploaded_paths[0]

df_raw = pd.read_excel(path)
df_raw.columns = [str(c).strip() for c in df_raw.columns]

# 자동 검출
detected = detect_columns(df_raw)
print("Detected time_col:", detected["time_col"])
print("Detected pm25_col:", detected["pm25_col"])
print("Detected metal_cols (first 10):", detected["metal_cols"][:10])
print("Detected numeric_cols (first 10):", detected["numeric_cols"][:10])

# 자동 검출 컬럼 확인
if not CONFIRM_DETECTED_COLUMNS:
    raise RuntimeError(
        "자동 검출된 컬럼 확인 후 맞다면 CONFIRM_DETECTED_COLUMNS=True로 변경 후 이 셀부터 다시 실행")

# 시간 컬럼 파싱
time_col = detected["time_col"]
pm25_col = detected["pm25_col"]
metal_cols = detected["metal_cols"]
numeric_cols = detected["numeric_cols"]

if time_col is not None:
    df_raw[time_col] = pd.to_datetime(df_raw[time_col], errors="coerce")

Detected time_col: Pump-Begin
Detected pm25_col: Conc(ug/m3)
Detected metal_cols (first 10): ['Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)', 'Sb(ng/m3)', 'Pb(ng/m3)']
Detected numeric_cols (first 10): ['Pump-Begin', 'Pump-End', 'MassResetTime', 'Conc(ug/m3)', 'Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)', 'Sb(ng/m3)']


In [85]:
# 전처리 전 설정
NEGATIVE_TO_NAN   = True   # 음수값을 NaN으로 치환
DROP_EMPTY_TARGET = True   # 대상 컬럼이 모두 NaN인 행 제거
APPLY_IQR_FILTER  = False   # IQR 기반 이상치 제거 적용
IQR_K             = 1.5    # IQR 배수(1.5 기본, 완화 2.0~3.0)

In [86]:
# 전제 조건
assert "df_raw" in globals(), "df_raw가 필요합니다."

# 대상 컬럼 정의 (PM2.5 + 금속)
target_cols = []
if "pm25_col" in globals() and pm25_col:
    target_cols.append(pm25_col)
if "metal_cols" in globals() and isinstance(metal_cols, (list, tuple)):
    target_cols += list(metal_cols)

# 실제 존재하는 컬럼만 남기기
target_cols = [c for c in target_cols if c in df_raw.columns]
if len(target_cols) == 0:
    raise ValueError("대상 컬럼을 찾지 못했습니다.")

# 복사본 생성
df_clean = df_raw.copy()

In [87]:
# 대상 컬럼을 숫자로 변환한 프레임 준비
num_targets = df_clean[target_cols].apply(pd.to_numeric, errors="coerce")

# 음수값 NaN 처리
neg_before = (num_targets < 0).sum().sum()
if NEGATIVE_TO_NAN:
    df_clean[target_cols] = num_targets.mask(num_targets < 0, np.nan)
else:
    df_clean[target_cols] = num_targets

# 대상 컬럼이 전부 NaN인 행 제거
rows_before_drop_empty = len(df_clean)
if DROP_EMPTY_TARGET:
    df_clean = df_clean.dropna(subset=target_cols, how="all").reset_index(drop=True)
rows_dropped_empty = rows_before_drop_empty - len(df_clean)

rows_dropped_iqr = 0

In [88]:
# 시간 컬럼 정렬
if "time_col" in globals() and time_col in df_clean.columns:
    df_clean[time_col] = pd.to_datetime(df_clean[time_col], errors="coerce")
    df_clean = df_clean.sort_values(time_col).reset_index(drop=True)

# 음수 잔존 여부 확인
neg_after = (df_clean[target_cols].apply(pd.to_numeric, errors="coerce") < 0).sum().sum()

print("=== Cleaning Summary ===")
print(f"Target columns (n={len(target_cols)}): {target_cols[:6]}{' ...' if len(target_cols)>6 else ''}")
print(f"Negatives (before -> after): {int(neg_before)} -> {int(neg_after)}")
print(f"IQR dropped rows (k={IQR_K}): {rows_dropped_iqr}")
print(f"Empty-target dropped rows: {rows_dropped_empty}")
print(f"Shape: raw {df_raw.shape} -> clean {df_clean.shape}")

=== Cleaning Summary ===
Target columns (n=8): ['Conc(ug/m3)', 'Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)'] ...
Negatives (before -> after): 518 -> 0
IQR dropped rows (k=1.5): 0
Empty-target dropped rows: 1
Shape: raw (517, 11) -> clean (516, 11)


In [90]:
# 전처리 데이터 저장

df_clean.to_excel("202501_clean.xlsx", index=False)

files.download("202501_clean.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>